#### Initialisation (have to run it every time)

Install packages

In [ ]:
# data parsing and working with dataset
# !pip install beautifulsoup4 
# !pip install requests
# !pip install transformers
# !pip install pandas

!pip install openpyxl --upgrade --ignore-installed openpyxl
from bs4 import BeautifulSoup
import requests as rq
import pandas as pd
import numpy as np
from google.colab import drive
import torch

# import zipfile

# for text preprocessing
import re
from IPython.core.display import clear_output

# for formatting
from datetime import datetime, timedelta

# for visualization
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

     |████████████████████████████████| 242 kB 4.1 MB/s 


In [ ]:
# Import Google Drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/HSE/Project 2021'

# !unzip /content/drive/MyDrive/ML_IAD/iad-deep-learning-texts-2021.zip
# press_releases = pd.read_excel('cbr.xlsx')

Mounted at /content/drive
/content/drive/MyDrive/HSE/Project 2021


In [ ]:
# Additional functions


def delete_multiple_element(list_object, indices):
    '''
    delete multiple objects with certain indeces from the list
    from https://thispointer.com/python-remove-elements-from-list-by-index/#4
    '''
    indices = sorted(indices, reverse=True)
    for idx in indices:
        if idx < len(list_object):
            list_object.pop(idx)


def get_date(date, df=True, file_type=''):
    '''
    Tranform date from the cbr.ru into date format
    '''
    if df==True:
        month_list = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня', 'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря']
        date_list = date.split()
        if file_type == 'Пресс-релиз':
            return datetime(int(date_list[2]), month_list.index(date_list[1])+1, int(date_list[0]), 13, 30)
        elif file_type == 'Пресс-конференция':
            return datetime(int(date_list[2]), month_list.index(date_list[1])+1, int(date_list[0]), 15, 00)
        else:
            return datetime(int(date_list[2]), month_list.index(date_list[1])+1, int(date_list[0]), 10, 00) # the time does not matter for other types of documents - it is only important to get the news before the publication
    else:
        date_list = str(date).split()[0].split('-')
        return (str(date_list[2]) + '.' + 
                str(date_list[1]) + '.' + 
                str(date_list[0])) # dd.mm.yyyy

def get_date_and_time(date, time):
    month_list = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня', 'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря']
    date_list = date.split()
    time_list = time.split(':')
    return datetime(int(date_list[2]), month_list.index(date_list[1])+1, int(date_list[0]), int(time_list[0]), int(time_list[1]))



def unpack_list(mylist):
    '''
    flatten the list of lists
    '''
    return [x for l in mylist for x in l]




### News Parsing

Notebooks used:

https://colab.research.google.com/drive/1qFlX5rmkAnF5O6yhyqht-njmdRFJjeCf?usp=sharing

https://drive.google.com/drive/folders/1EpFItxXDLPEdAgR8Z9wdD7eqf7Dcmmrw

In [ ]:
#@title Class rbc_parser { display-mode: "form" }

import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime, timedelta
from IPython import display

class rbc_parser:
    def __init__(self):
        pass
    
    
    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями
        """
        url = 'https://www.rbc.ru/v10/search/ajax/?\
        project={0}&\
        category={1}&\
        dateFrom={2}&\
        dateTo={3}&\
        offset={4}&\
        limit={5}&\
        query={6}&\
        material={7}'.format(param_dict['project'],
                            param_dict['category'],
                            param_dict['dateFrom'],
                            param_dict['dateTo'],
                            param_dict['offset'],
                            param_dict['limit'],
                            param_dict['query'],
                            param_dict['material'])
        
        return url
    
    
    def _get_search_table(self, param_dict: dict,
                          includeText: bool = True) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        
        includeText: bool
        ### Если True, статьи возвращаются с текстами
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        # soup = bs(request.text)
        search_table = pd.DataFrame(r.json()['items'])
        if includeText and not search_table.empty:
            get_text = lambda x: self._get_article_data(x['fronturl'])
            search_table[['overview', 'text']] = search_table.apply(get_text,
                                                                    axis=1).tolist()
            
        return search_table
    
    
    def _get_article_data(self, url: str):
        """
        Возвращает описание и текст статьи по ссылке
        """
        r = rq.get(url)
        soup = bs(r.text, features="lxml") # features="lxml" чтобы не было warning
        div_overview = soup.find('div', {'class': 'article__text__overview'})
        if div_overview:
            overview = div_overview.text.replace('<br />','\n').strip()
        else:
            overview = None
        p_text = soup.find_all('p')
        if p_text:
            text = ' '.join(map(lambda x:
                                x.text.replace('<br />','\n').strip(),
                                p_text))
        else:
            text = None
        
        return overview, text 
    
    def get_articles(self,
                     param_dict,
                     time_step = 7,
                     save_every = 5,
                     out: pd.DataFrame = None) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса 
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%d.%m.%Y')
        dateTo = datetime.strptime(param_copy['dateTo'], '%d.%m.%Y')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')
        
        # global out ||| не знаю как out сделать
        if out is None:
            out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime("%d.%m.%Y")
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime("%d.%m.%Y")
            print('Parsing articles from ' + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime("%d.%m.%Y")
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("checkpoint_table.xlsx")
                print('Checkpoint saved')
                save_counter = 0
            
        out.to_excel("table_of_articles_{}.xlsx".format(
            datetime.strptime(param_dict['dateFrom'],
                              '%d.%m.%Y').strftime('%Y')))
        print('Finish')
        # res.to_csv("output.csv", sep = ';')
        
        return out
            


In [ ]:
#@title Class lentaRu_parser { display-mode: "form" }

import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime, timedelta
from IPython import display


class lentaRu_parser:
    def __init__(self):
        pass
    
    
    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])
        
        return url


    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])
        for datetype in ['lastmodtime', 'modified', 'pubdate']:
            search_table[datetype] = [datetime.fromtimestamp(date) for date in search_table[datetype]]
        
        return search_table

    
    def get_articles(self,
                     param_dict,
                     time_step = 37,
                     save_every = 5,
                     out: pd.DataFrame = None) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса 
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')
        
        # global out ||| не знаю как out сделать
        if out is None:
            out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print('Parsing articles from '\
                  + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("checkpoint_table.xlsx")
                print('Checkpoint saved')
                save_counter = 0
            
        out.to_excel("table_of_articles_{}.xlsx".format(
            datetime.strptime(param_dict['dateFrom'],
                              '%Y-%m-%d').strftime('%Y')))
        print('Finish')
        # res.to_csv("output.csv", sep = ';')
        
        return out

In [ ]:
#@title # Search Parameters { display-mode: "form" }

use_parser = "LentaRu" #@param ["РБК", "LentaRu"]

#@markdown ## РБК
#@markdown query \\
#@markdown project - RBC projects \\
#@markdown category \\
#@markdown material \\
#@markdown dateFrom \\
#@markdown dateTo \\
#@markdown offset - shift of the query response \\
#@markdown limit - query limit (max 100) \\
#@markdown #### If parameter specifications is not needed - the field should be left empty

query = ''                   #@param {type: "string"}
project = "rbcnews" #@param ["rbcnews", "rbctv", "rbcstyle", "sport", "realty", "crypto", "autonews", "quote", "bc3", "trends"] {allow-input: true}
category = "TopRbcRu_finances" #@param ["TopRbcRu_economics", "TopRbcRu_auto", "TopRbcRu_business", "TopRbcRu_money", "TopRbcRu_realty", "TopRbcRu_society", "TopRbcRu_politics", "TopRbcRu_own_business", "TopRbcRu_specials", "TopRbcRu_technology_and_media", "TopRbcRu_finances"] {allow-input: true}
material = "" #@param ["video", "quiz", "interview", "research", "card", "opinion", "multimedia", "short_news", "olympics_online", "online", "investigation", "rating", "article_specproject", "article", "story"] {allow-input: true}
dateFrom = '2013-09-01'         #@param {type: "date"}
dateTo = "2013-09-28"           #@param {type:"date"}
offset = 0                      #@param {type:"slider", min:0, max:100, step:1}
limit = 100 #@param {type:"slider", min:0, max:100, step:1}

#@markdown ---

if use_parser == "РБК":
    param_dict = {'query'   : query, 
                  'project' : project,
                  'category': category,
                  'dateFrom': datetime.
                  strptime(dateFrom, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'dateTo'  : datetime.
                  strptime(dateTo, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'offset'  : str(offset),
                  'limit'   : str(limit),
                  'material': material}

#@markdown ## LentaRu
#@markdown query - поисковой запрос (ключевое слово) \\
#@markdown offset - cмещение поисковой выдачи \\
#@markdown size - количество статей. Ограничено время запроса, точного лимита нет. 1000 работает почти всегда \\
#@markdown sort - сортировка по дате: (2) - по убыванию, (3) - по возрастанию; по релевантности (1) \\
#@markdown title_only - точная фраза в заголовке (1) \\
#@markdown domain - ? \\
#@markdown material - материалы: Все материалы (0). Новость (1) \\
#@markdown block - рубрика: Экономика (4). Все рубрики (0) \\
#@markdown dateFrom - с даты \\
#@markdown dateTo - по дату \\
#@markdown #### Чтобы не специфировать параметр, оставляем поле пустым

query = ''                   #@param {type: "string"}
offset = 0 #@param {type:"integer"}
size = 1000 #@param {type:"slider", min:0, max:1000, step:1}
sort = "2" #@param ["1", "2", "3"] {allow-input: true}
title_only = "0" #@param ["0", "1"]
domain = "1" #@param ["1"] {allow-input: true}
material = "0" #@param ["0", "1", "2", "3", "4"] {allow-input: true}
bloc = "4" #@param ["0", "1", "2", "3", "4"] {allow-input: true}
dateFrom = '2013-09-01'         #@param {type: "date"}
dateTo = "2013-09-28"           #@param {type:"date"}

#@markdown ---

if use_parser == "LentaRu":
    param_dict = {'query'     : query, 
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material, 
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)


LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2013-09-01', 'dateTo': '2013-09-28', 'sort': '2', 'title_only': '0', 'type': '0', 'bloc': '4', 'domain': '1'}


In [ ]:
assert use_parser == "РБК"
parser = rbc_parser()
tbl = parser._get_search_table(param_dict, includeText = False)
# tbl.head()

In [ ]:
tbl = parser.get_articles(param_dict=param_dict)

In [ ]:
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict)
print(len(tbl.index))
tbl.head()

### News Pre-Processing

In [ ]:
# Tokenising

!pip install pymorphy2
import pymorphy2


import nltk
import string
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['также', 'это'])
morph = pymorphy2.MorphAnalyzer()

clear_output()

def delete_stop_words(text):
    return ' '.join([morph.parse(word)[0].normal_form for word in word_tokenize(text) if word not in russian_stopwords])

In [ ]:
# @title Importing News and Previews (from Google Drive)

#@markdown If we want to filter them and process. Otherwise -- skip a few cells. 

previews = pd.read_excel('cbr_previews.xlsx').drop(['Unnamed: 0'], axis = 1) # re-import data because we do not want to get it from the site every single time
previews['date'] = pd.to_datetime(previews['date']-timedelta(hours=3), utc=True)
news = pd.read_excel('news/news_preprocessed.xlsx').drop(['Unnamed: 0'], axis = 1) 

# КОРОЧЕ, СЕЙЧАС ОСТАВЛЯЮ ТОЛЬКО РБК, ТК ЛЕНТА ЗАКРЫВАЕТ МНЕ ВСЮ ИНФУ!!!
news = news[(news['source']=='rbc_econ') | (news['source']=='rbc_finance')].reset_index(drop=True)

# rbc_finance = pd.read_excel('news/rbc_finance_2012_2021.xlsx').drop(['Unnamed: 0'], axis = 1)
# rbc_econ = pd.read_excel('news/rbc_econ_2013_2021.xlsx').drop(['Unnamed: 0'], axis = 1)
# lenta_econ = pd.read_excel('news/lenta_news_2013_2021.xlsx').drop(['Unnamed: 0'], axis = 1)

# rbc_finance['source'] = 'rbc_finance' 
# rbc_econ['source'] = 'rbc_econ' 
# lenta_econ['source'] = 'lenta_econ' 

# leave out excessive columns and edit date format, sort on date
# news = pd.concat([rbc_finance[['publish_date', 'source', 'title', 'overview', 'text']].rename(columns= {'publish_date': 'date'}), 
#                   rbc_econ[['publish_date', 'source', 'title', 'overview', 'text']].rename(columns= {'publish_date': 'date'}), 
#                   lenta_econ[['pubdate', 'source', 'title', 'rightcol', 'text']].rename(columns= {'pubdate': 'date', 'rightcol': 'overview'})], ignore_index=True)
# news['date'] = pd.to_datetime(news['date'], utc=True)
# news = news[[type(i)==str for i in news['text']]]
# news = news.sort_values('date').reset_index(drop=True)
# news['date'] = [datetime.strftime(i, '%Y-%m-%d %H:%M:%S') for i in news['date']]
# news.to_excel('news/all_news_last.xlsx')
# already have done everything, import the result

# news = pd.read_excel('news/news.xlsx').drop(['Unnamed: 0'], axis = 1)
# news['date'] = pd.to_datetime(news['date'], format='%Y-%m-%d %H:%M:%S%z')

In [ ]:
news

,date,title,overview,text,source,preview_index,preprocessed
0,2013-10-08 04:32:25,"СМИ подсчитали, во сколько сборной России обой...",Стоимость аренды крытого конькобежного центра ...,час аренды. Для санного спорта и бобслея оплат...,rbc_econ,67,час аренда санный спорт бобслей оплата один сп...
1,2013-10-08 08:19:31,"Молодежная безработица: ""потерянное поколение""...",Всем любителям творчества Эриха Марии Ремарка ...,"С того времени прошел почти век, ужасы кровопр...",rbc_econ,67,время пройти век ужас кровопролитный сражение ...
2,2013-10-08 13:31:32,Эксперт: Российский бюджет может заработать тр...,Наполнить государственную казну триллионами ру...,"По мнению эксперта, благодаря этой программе г...",rbc_econ,67,мнение эксперт благодаря программа государство...
3,2013-10-09 10:27:15,Экономическое сотрудничество РФ и Украины пост...,Премьер-министр Украины Николай Азаров хотел б...,"""Убежден, что мы можем воспользоваться опытом ...",rbc_econ,67,убеждённый мочь воспользоваться опыт украинско...
4,2013-10-09 11:12:50,Вестники разлуки: Россия может запретить поста...,Россельхознадзор может запретить поставку тюль...,"Он также отметил, что претензии озвучивались п...",rbc_econ,67,отметить претензия озвучиваться последний неск...
...,...,...,...,...,...,...,...
2141,2021-12-17 08:10:31,Кабмину предложили расширить маткапитал на пок...,"Авторы инициативы считают, что ограничения на ...",Правительству предложили разрешить использоват...,rbc_econ,0,правительство предложить разрешить использоват...
2142,2021-12-17 10:30:25,ЦБ повысил ставку до максимума с 2017 года. И ...,"ЦБ повысил ставку на 1 п.п., до 8,5%. Это може...",Совет директоров Банка России на заседании 17 ...,rbc_finance,0,совет директор банк россия заседание декабрь п...
2143,2021-12-17 11:08:51,Как менялась ключевая ставка ЦБ за последние г...,ЦБ на последнем в этом году заседании поднял к...,,rbc_finance,0,NaN
2144,2021-12-17 11:26:37,Бизнес за решеткой: кто из предпринимателей по...,17 декабря в Москве задержали экс-владелицу ко...,17 декабря в Москве задержали экс-владелицу ко...,rbc_econ,0,декабрь москва задержать эксвладелица компания...


In [ ]:
#@title Filtering and Preprocessing the News

#@markdown **Only if we decided to run the previous one!**

#@markdown We need to define the length of the time window before the preview release that we will use to get the news (that we need to proxy market expectations). Also we need to filter these news on some word to be ore specific about what news we accept as market predictions.
query = ''                   #@param {type: "string"}
days = 3 #@param {type:"slider", min:1, max:7, step:1}
#@markdown The process takes some time. ПОКА САМОЕ МАЛЕНЬКОЕ КОЛИЧЕСТВО ДНЕЙ (ПРИ РЕЛЕВАНТНЫХ НОВОСТЯХ) - 4. ОСТАВИЛА ТОЛЬКО НОВОСТИ РБК И КОТОРЫЕ УПОМИНАЮТ ЦБ






# filter news
X = news.copy().drop(['preview_index'], axis = 1).reset_index(drop=True)
X['date'] = pd.to_datetime(X['date'], utc='UTC')
preview_index = [[j for j in range(len(previews)) if (X['date'][i] >= (previews['date'][j]-timedelta(days=days))) and (X['date'][i] < previews['date'][j]-timedelta(hours=5))] for i in range(len(X))]  
preview_index = [id[0] if len(id) > 0 else np.nan for id in preview_index]    
X['preview_index'] = preview_index          
X = X.dropna().reset_index(drop=True)



# here we define what we want to find in the news. I want to use every news that has reference to CBR in the text (mostly, because there is at least 1 such news before every preview)

X = X[[query in X['preprocessed'][i] for i in range(len(X['preprocessed']))]].reset_index(drop=True)
# X['text'] = [i[i.rfind('/')+1:] if i.rfind('/')!=-1 else i for i in X['text']]




# Simple check that we have at least 1 article for each preview.
print("Indeces of the previews that are not backed up by news:", *[j for j in range(len(previews)) if j not in set(X['preview_index'])]) if len([j for j in range(len(previews)) if j not in set(X['preview_index'])]) != 0 else print('All CBR previews are backed with news articles!')




# preprocess news
# X['preprocessed'] = [delete_stop_words(re.sub(r'[^\w\s]+|[\d]+', r'',X['text'][i].lower()).strip()) for i in range(len(X['text']))]

X.head()

All CBR previews are backed with news articles!


,date,title,overview,text,source,preprocessed,preview_index
0,2013-10-11 08:55:27+00:00,Аналитики: С новыми акционерами Tele2 сможет п...,"Появление среди акционеров ""Tele2 Россия"" стру...","""Продажа 50% акций могла произойти только стру...",rbc_econ,продажа акция мочь произойти структура близко ...,67.0
1,2013-10-11 10:18:11+00:00,"А.Лукашенко: С.Керимову плевать на ""Уралкалий""...","Ключевого акционера ""Уралкалия"" Сулеймана Кери...","""Ни один государственный ответственный человек...",rbc_econ,государственный ответственный человек поступит...,67.0
2,2013-10-11 10:55:09+00:00,"Главе ""Уралкалия"" поменяли статью обвинения на...","Главе ""Уралкалия"" Владиславу Баумгертнеру пере...","""Он (Баумгертнер. - Примеч. РБК) выпал в осадо...",rbc_econ,баумгертнер примеч рбк выпасть осадка понять д...,67.0
3,2013-10-11 13:53:19+00:00,"А.Лукашенко предложил В.Путину купить долю в ""...",Президент Белоруссии Александр Лукашенко отмет...,"""Путину я сказал: не затягивайте с принятием р...",rbc_econ,путин сказать затягивать принятие решение край...,67.0
4,2013-11-05 12:56:36+00:00,Минэнерго Украины: Начато погашение долга пере...,"""Нефтегаз Украины"" начал погашение долга перед...","""Платежи уже пошли вчера. Это будет в достаточ...",rbc_econ,платёж послать вчера достаточный объём закрыть...,66.0


In [ ]:
#@title If we've decided to not run the previous cells and to work with already preprocessed data

previews = pd.read_excel('cbr_previews.xlsx').drop(['Unnamed: 0'], axis = 1)
X = pd.read_excel('news_for_expectations.xlsx').drop(['Unnamed: 0'], axis = 1)
X.head()
X['text'] = [i[i.rfind('/')+1:] if i.rfind('/')!=-1 else i for i in X['text']]
print("Indeces of the previews that are not backed up by news:", *[j for j in range(len(previews)) if j not in set(X['preview_index'])]) if len([j for j in range(len(previews)) if j not in set(X['preview_index'])]) != 0 else print('All CBR previews are backed with news articles!')
X['preprocessed'] = [delete_stop_words(re.sub(r'[^\w\s]+|[\d]+', r'',X['text'][i].lower()).strip()) for i in range(len(X['text']))]
X.head()

# news = pd.read_excel('news/news_preprocessed.xlsx').drop(['Unnamed: 0'], axis = 1) 
# news['date'] = pd.to_datetime(news['date'], format='%Y-%m-%d %H:%M:%S%z')

In [ ]:
# X_rbc = X[X['source'] != 'lenta_econ']  # 16 апреля

In [ ]:
# X_rbc.to_excel('only_rbc_news_upd16042022.xlsx')  # 16 апреля

In [ ]:
#@markdown Simple check that we have at least 1 article for each preview.
print("Indeces of the previews that are not backed up by news:", *[j for j in range(len(previews)) if j not in set(X_rbc['preview_index'])]) if len([j for j in range(len(previews)) if j not in set(X_rbc['preview_index'])]) != 0 else print('All CBR previews are backed with news articles!')

All CBR previews are backed with news articles!
